In [151]:
import dataloader
import numpy as np
import pandas as pd
import plotly.express as px
import geopy
from schemas import Trip
import datetime as dt

In [152]:
trips = dataloader.TripsLoader('03-07-2022')

In [153]:
#Machine numbers present for given day
trips._machines.keys()

dict_keys([20, 13, 25, 11, 46, 37, 52, 15, 19, 22, 26, 5, 44, 16, 30, 36, 23, 1, 38, 24, 41, 8, 4, 50, 49])

In [154]:
machine_nb = 20 #Choice of machine number
machine_nb_info = trips._machines[machine_nb]#[machine for machine in trips._machines if machine.machine_id==machine_nb][0]
all_trips_for_machine = machine_nb_info.trips

In [155]:
#Look at actual trip recordings
start_end_each_trip_dict = [dict(Start=trips.start_date, End=trips.end_date) for trips in all_trips_for_machine]
df_pyplot = pd.DataFrame(start_end_each_trip_dict)

fig = px.timeline(df_pyplot, x_start="Start", x_end="End")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()


In [156]:
# Collect all positions throughout day
# Initialize first data point as load spot of trip 1 => Start trip 1
# Iteratively run next points through an algo that decides if load or dump, and start new trip each time load is decided (either explicitly or only implicitly)

In [157]:
all_trips_for_machine

[Trip(trip_id='6225a07a03d57f0000de776a', load='Stone', quantity=16.0, positions=[Position(lat=59.94648840546584, lon=10.3860295631773, uncertainty=8.942957828638438, timestamp=Timestamp('2022-03-07 06:04:41.869000+0000', tz='UTC')), Position(lat=59.94648630530464, lon=10.386050682931844, uncertainty=9.52108084696358, timestamp=Timestamp('2022-03-07 06:05:00.868000+0000', tz='UTC')), Position(lat=59.946474177350765, lon=10.386081239842635, uncertainty=8.552684282533649, timestamp=Timestamp('2022-03-07 06:05:37.866000+0000', tz='UTC')), Position(lat=59.94647847397913, lon=10.386037727373054, uncertainty=8.830488869178405, timestamp=Timestamp('2022-03-07 06:06:36.863000+0000', tz='UTC')), Position(lat=59.946482425033295, lon=10.386001831711695, uncertainty=9.107994890565934, timestamp=Timestamp('2022-03-07 06:06:38.863000+0000', tz='UTC')), Position(lat=59.9464939728644, lon=10.385954199579334, uncertainty=9.335028777962632, timestamp=Timestamp('2022-03-07 06:06:41.863000+0000', tz='UTC'

In [158]:
len(all_trips_for_machine)

22

In [159]:
all_positions_and_times = [trip.positions[0] for trip in all_trips_for_machine]

In [160]:
all_positions_and_times

[Position(lat=59.94648840546584, lon=10.3860295631773, uncertainty=8.942957828638438, timestamp=Timestamp('2022-03-07 06:04:41.869000+0000', tz='UTC')),
 Position(lat=59.94647050624484, lon=10.386122472055275, uncertainty=4.015248289087832, timestamp=Timestamp('2022-03-07 06:28:53.850000+0000', tz='UTC')),
 Position(lat=59.9464729347381, lon=10.386150818972316, uncertainty=9.124901331480167, timestamp=Timestamp('2022-03-07 06:52:39.879000+0000', tz='UTC')),
 Position(lat=59.9464546359841, lon=10.386019780221217, uncertainty=3.610763961609692, timestamp=Timestamp('2022-03-07 07:18:09.871000+0000', tz='UTC')),
 Position(lat=59.9464757455572, lon=10.386244496249509, uncertainty=9.930720895503384, timestamp=Timestamp('2022-03-07 07:39:02.838000+0000', tz='UTC')),
 Position(lat=59.94649191215338, lon=10.38622515378083, uncertainty=10.47376852455024, timestamp=Timestamp('2022-03-07 08:03:18.849000+0000', tz='UTC')),
 Position(lat=59.946503050325454, lon=10.386252346719212, uncertainty=11.402

In [161]:
all_positions_and_times = sorted(all_positions_and_times, key=lambda Position: Position.timestamp) #Is it actually sorted now

In [162]:
##Algorithm - first Position is first recording of trip 1.
## Have some sort of placeholder for current trip
## Add location for predicted load spot
## Add new positions to the trip
## If predicted dump -> Record dump position
## Add new positions to the trip
## If predicted load -> Stop trip
## Reiterate until EOL all positions
##Should maybe throw error/warning if times is not sorted etc. or locations jumps unreasonably much

##Possible logic for dump and load
## - Minimal movement around predicted time
## - In vicinity of spots of loading and dumping -> should be made available prior to day -> Or decide based on yesterdays spot -> i.e. within 500m from last days spot -> can be prompted by driver if they are going to new location

In [163]:
automated_trips = []
temp_positions = []
trip_counter = 0
predicted_dump = False #Must have predicted dump for possible load prediction
speed_threshold = 10

#Initialize day
temp_positions.append(all_positions_and_times[0])

temp_load_pos = all_positions_and_times[0]
temp_dump_pos = temp_load_pos #Just to initialize


for pos in all_positions_and_times[1:]:
    temp_positions.append(pos)

    #Some logic, e.g. if movement less than 3m/s since last, predict spot
    current_time = pos.timestamp
    current_lat_lon = (pos.lat, pos.lon)
    prev_time = temp_positions[-2].timestamp
    prev_lat_lon = (temp_positions[-2].lat,temp_positions[-2].lon)
    seconds_gone = (current_time.to_pydatetime()-prev_time.to_pydatetime()).total_seconds()
    meters_driven = geopy.distance.geodesic(prev_lat_lon, current_lat_lon).m

    if meters_driven/seconds_gone < speed_threshold and predicted_dump == False:
        #Reached predicted dump spot
        predicted_dump = True
        temp_dump_pos = pos

    if meters_driven/seconds_gone < speed_threshold and predicted_dump == True:
        #Reached predicted load spot, trip is over
        
        temp_trip_id = str(trip_counter) #Placeholder

        automated_trips.append(Trip(trip_id=temp_trip_id, load='Stone', quantity=10.0, positions = temp_positions, dump_latlon=[temp_dump_pos.lat, temp_dump_pos.lon], load_latlon=[temp_load_pos.lat, temp_load_pos.lon]))

        #Reinitialize/increment variables
        temp_load_pos = pos
        predicted_dump = False
        trip_counter += 1
        temp_positions = [pos]





In [164]:
len(automated_trips)

21

In [165]:
automated_trips

[Trip(trip_id='0', load='Stone', quantity=10.0, positions=[Position(lat=59.94648840546584, lon=10.3860295631773, uncertainty=8.942957828638438, timestamp=Timestamp('2022-03-07 06:04:41.869000+0000', tz='UTC')), Position(lat=59.94647050624484, lon=10.386122472055275, uncertainty=4.015248289087832, timestamp=Timestamp('2022-03-07 06:28:53.850000+0000', tz='UTC'))], dump_latlon=(59.94647050624484, 10.386122472055275), load_latlon=(59.94648840546584, 10.3860295631773), latlons=[(59.94648840546584, 10.3860295631773), (59.94647050624484, 10.386122472055275)], length=0.005562440053994566, duration=24.199683333333333),
 Trip(trip_id='1', load='Stone', quantity=10.0, positions=[Position(lat=59.94647050624484, lon=10.386122472055275, uncertainty=4.015248289087832, timestamp=Timestamp('2022-03-07 06:28:53.850000+0000', tz='UTC')), Position(lat=59.9464729347381, lon=10.386150818972316, uncertainty=9.124901331480167, timestamp=Timestamp('2022-03-07 06:52:39.879000+0000', tz='UTC'))], dump_latlon=(5

In [166]:
#Look at predicted trip recordings
start_end_each_trip_dict_automated = [dict(Start=trips.positions[0].timestamp, End=trips.positions[-1].timestamp) for trips in automated_trips]
df_pyplot_automated  = pd.DataFrame(start_end_each_trip_dict_automated)

fig = px.timeline(df_pyplot_automated, x_start="Start", x_end="End")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

In [170]:
from plotly.subplots import make_subplots

fig_actual = px.timeline(df_pyplot, x_start="Start", x_end="End")
fig_actual.update_yaxes(autorange="reversed")

fig_automated = px.timeline(df_pyplot_automated, x_start="Start", x_end="End")
fig_automated.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up

fig_sub = make_subplots(rows=2, shared_xaxes=True)
fig_sub.append_trace(fig_actual['data'][0], row=1, col=1)
fig_sub.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.2, showarrow=False,
                   text="<b>Actual recorded trips</b>", row=1, col=1)
fig_sub.append_trace(fig_automated['data'][0], row=2, col=1)
fig_sub.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.2, showarrow=False,
                   text="<b>Autmated recorded trips</b>", row=2, col=1)

fig_sub.update_xaxes(type='date')
fig_sub.show()